In [1]:
import numpy as np
import sys, os
import time
import glob
from registration_3d import *
from cartesian import *

In [2]:
data_dir = 'DATA/'
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']
runtype = ['Debug' ,'Debug' ,'Debug' ,'Debug' ,'Debug' ,'Debug' ,'Unknown','Unknown','Unknown']
pa = "PA3"
filename = '/Problem3MeshFile.sur'

In [3]:
def read_mesh( data_dir , filename ):
# get the coordinate of points
# get the vertices of each triangle
    mesh = glob.glob( data_dir + filename )
    mesh_file = open( mesh[0] , "r")
    mesh_lines = mesh_file.read().splitlines()
    mesh_data = []
    
    for num in range(len(mesh_lines)):
        mesh_data.append( mesh_lines[num].split(' ') )
        for i in range(len( mesh_data[-1])):
            mesh_data[-1][i] = mesh_data[-1][i].strip() # to remove space and tabs
    Np = int(mesh_data[0][0])
    mesh_data = mesh_data[1:]
    
    mesh_points = np.asarray(mesh_data[0:Np]).astype(float)
    mesh_data = mesh_data[Np:]
    
    Ntriangles = int(mesh_data[0][0])
    mesh_data = mesh_data[1:]
    mesh_vertices = np.asarray(mesh_data[0:Ntriangles]).astype(int)
    mesh_data = mesh_data[Ntriangles:]
    
    return Np,mesh_points,Ntriangles,mesh_vertices

In [4]:
def read_body( data_dir , filename ):

# get the markers' positions and tip of a body
    body = glob.glob( data_dir + filename )
    body_file = open( body[0] , "r")
    body_lines = body_file.read().splitlines()
    body_data = []
    
    for num in range(len(body_lines)):
        body_data.append( body_lines[num].split() )
        for i in range(len( body_data[-1])):
            body_data[-1][i] = body_data[-1][i].strip() # to remove space and tabs
    Np = int(body_data[0][0])
    body_data = body_data[1:]
    
    body_points = np.asarray(body_data[0:Np]).astype(float)
    body_data = body_data[Np:]
    
    tip = body_data[0]
    tip = np.array(tip).astype(float)
    tip = tip.reshape(1,-1)
    return Np,body_points,tip


In [5]:
def read_sample( data_dir , filename , N_Ap , N_Bp ):
# get the markers' positions and tip of a body
# N_Ap is the number of marker in A
# N_Bp is the number of marker in B
    sample = glob.glob( data_dir + filename )
    sample_file = open( sample[0] , "r")
    sample_lines = sample_file.read().splitlines()
    sample_data = []
    
    for num in range(len(sample_lines)):
        sample_data.append( sample_lines[num].split(',') )
        for i in range(len( sample_data[-1])):
            sample_data[-1][i] = sample_data[-1][i].strip() # to remove space and tabs
    Np = int(sample_data[0][0])
    N_sample = int(sample_data[0][1])
    N_Dp = Np - N_Ap - N_Bp
    sample_data = sample_data[1:]
    sample_data = np.array(sample_data).astype(float)
    A_frames=[]
    B_frames=[]
    
    for i in range(N_sample):
        A_sample_points = np.asarray( sample_data[ i*Np : i*Np + N_Ap ] ).astype( float )
        A_frames.append( A_sample_points )
        
        B_sample_points = np.asarray( sample_data[ i*Np + N_Ap :  i*Np + N_Ap + N_Bp] ).astype( float )
        B_frames.append( B_sample_points )

    A_frames = np.array(A_frames).astype(float)
    B_frames = np.array(B_frames).astype(float)
        
    return A_frames , B_frames

In [6]:
def get_d_k(A_frames, A_points, A_tip, B_frames, B_points, B_tip):
    
    d_k = []
    for i in range(len(A_frames)):
        F_AK = registration_3d(A_points , A_frames[i])
        F_BK = registration_3d(B_points , B_frames[i])
        # F_AK is the transformation from A tool frame to optical frame
        # F_AK is the transformation from B tool frame to optical frame
        d_k_i = points_transform( Fi(F_BK) @ F_AK , A_tip)
        d_k.append(d_k_i)
    d_k = np.array(d_k).astype(float)
    d_k = d_k.reshape(-1,3)
    return d_k

In [7]:
def ProjectOnSegment(c,p,q):
# c is the point we want to find corresponding closest point
# p q are the start and the and of a segment

#according to https://ciis.lcsr.jhu.edu/lib/exe/fetch.php?media=courses:455-655:lectures:finding_point-pairs.pdf
# page 7
    c = np.array(c).astype(float)
    p = np.array(p).astype(float)
    q = np.array(q).astype(float)
    #print(np.dot( c-p , q-p ))
    #print(np.dot( q-p ,q-p ))
    Lambda = np.dot( c-p , q-p ) / np.dot( q-p ,q-p )
    Lambda = max(0, min(Lambda, 1))
    c_star = p + Lambda*(q-p)
    return c_star

In [8]:
def FindClosestPoint( a, p, q, r ):
    
    A = np.zeros((3,2),dtype = float )
    
    for i in range(3):
        A[i][0] = q[i] - p[i]
        A[i][1] = r[i] - p[i]
        
    B = a - p
    B = B.reshape(-1,1)
    X = np.linalg.lstsq(A, B, rcond=None)[0]
    
    Lambda = X[0]
    Miu = X[1]

    a = p + Lambda * (q - p) + Miu * (r - p)
    P_closest = np.zeros(3)

    if Lambda >= 0 and Miu>=0 and Lambda+Miu<=1:
        P_closest = a
    elif Lambda < 0:
        P_closest = ProjectOnSegment(a, r, p)
    elif Miu < 0:
        P_closest = ProjectOnSegment(a, p, q)
    elif Lambda + Miu > 1:
        P_closest = ProjectOnSegment(a, q, r)

    return P_closest


In [9]:
def matching(s_k , Np , mesh_points , Ntriangles , mesh_vertices ):
    Ns = s_k.shape[0]
    
    closest_p = []
    for i in range(Ns):
        mindist = np.finfo(np.float32).max
        P_closest = []
        for j in range(Ntriangles):
            p = mesh_points[mesh_vertices[j][0]] 
            q = mesh_points[mesh_vertices[j][1]]
            r = mesh_points[mesh_vertices[j][2]] 
            
            tmp = FindClosestPoint( s_k[i], p, q, r )

            dist = np.linalg.norm(s_k[i] - tmp)
            if mindist > dist:
                mindist = dist
                P_closest = tmp
        closest_p.append(P_closest)
                
    return closest_p

In [10]:
for num in range(1):
    data_type = num
    letter = num
    Np,mesh_points,Ntriangles,mesh_vertices = read_mesh(data_dir , '/Problem3MeshFile.sur')
    N_Ap,A_points,A_tip = read_body(data_dir , '/Problem3-BodyA.txt')
    N_Bp,B_points,B_tip = read_body(data_dir , '/Problem3-BodyB.txt')
    
    sample_dir = '/PA3-' + letters[num] + '-' + runtype[num] + '-SampleReadingsTest.txt'
    A_frames , B_frames = read_sample(data_dir, sample_dir, N_Ap , N_Bp)
    
    d_k = get_d_k(A_frames, A_points, A_tip, B_frames, B_points, B_tip)
    F_reg = np.eye(4)
    #in PA3 F_reg = identity matrix
    s_k = points_transform(F_reg , d_k) 
    
    # c_k_i are the closest points on the surface to s_k_i 
    c_k = matching(s_k , Np , mesh_points , Ntriangles , mesh_vertices )

ValueError: operands could not be broadcast together with shapes (3,) (0,) 

In [ ]:
#if __name__ == '__main__':
#    main()

In [ ]:
len(d_k)

In [ ]:
A_tip.shape

In [ ]:
c_k